In [1]:
# Import required modules
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [2]:
# Read in data from which image link will be created
link_df = pd.read_csv("_Bucket 3.1 - Sheet1.csv")

# Insert an inmdicator column for tracking
link_df["ind"] = range(1, link_df.shape[0]+1)

# Seprate only shutterstock
shutter_only = link_df[link_df["Image Name"].str.contains("shutterstock", na=False, case=False)]

# Create link to make requests to
shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")

<ipython-input-2-c00094b9af85>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")
<ipython-input-2-c00094b9af85>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")


In [3]:
# This function scrapes image info from a link
def scrape_image_info(url):
    HEADERS = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
    r = requests.get(url, headers=HEADERS)
    s = BeautifulSoup(r.text, "lxml")
    
    # Scrape keywords
    try:
        keywords = [t.text for t in s.find(class_="C_a_03061").find_all("a")]
    except:
        keywords = "na"
    
    # Scrape category
    try:
        category = s.find(class_="m_v_89d62 font-body-static-sm m_p_51037").text.replace("\xa0", "").replace("Categories:", "").strip()
    except:
        category = "na"
    
    # Scrape title
    try:
        title = s.find(class_="m_b_d59a1 font-headline-base").text.strip()
    except:
        title = "na"
    
    # Scrape image source url
    try:
        source_link = s.find(class_="m_t_a99ec").img["src"]
    except:
        source_link = "na"
    
    
    # Create a temp df
    temp_df = pd.DataFrame({
        "title":title,
        "category":category,
        "keywords":[keywords],
        "created_link":url,
        "source_link":source_link})
    
    return temp_df

In [6]:
# Apply the function and scrape all link image info
with ProcessPoolExecutor(max_workers=6) as ex:
    scraped_df = pd.concat(list(ex.map(scrape_image_info, shutter_only.created_link.unique()))).reset_index(drop=True)

scraped_df.head(10)

,description,category,keywords,created_link,source_link
0,Elderly asian woman suffering with parkinson's...,"People,Healthcare/Medical","[disease, parkinson, adult, aging, alzheimer, ...",https://www.shutterstock.com/image-photo/14110...,https://image.shutterstock.com/image-photo/eld...
1,Male Scientist Working on a Laptop Computer an...,"People,Science","[food, test, health, analysis, analyzing, arti...",https://www.shutterstock.com/image-photo/19194...,https://image.shutterstock.com/image-photo/mal...
2,DNA structure,"Backgrounds/Textures,Healthcare/Medical","[dna, strand, test, 3d, biotechnology, blue, h...",https://www.shutterstock.com/image-photo/23112...,https://image.shutterstock.com/image-illustrat...
3,Scientist or medical in lab coat holding test ...,"Science,Healthcare/Medical","[medical, science, research, scientist, techno...",https://www.shutterstock.com/image-photo/16873...,https://image.shutterstock.com/image-photo/sci...
4,Woman using DNA helix interface hologram,"People,","[dna, doctor, helix, hologram, lab, technology...",https://www.shutterstock.com/image-photo/11770...,https://image.shutterstock.com/image-photo/wom...
5,Female scientist working in modern lab. Doctor...,"Science,Healthcare/Medical","[research, lab, healthcare, science, scientifi...",https://www.shutterstock.com/image-photo/16871...,https://image.shutterstock.com/image-photo/fem...
6,"Closeup portrait, young friendly scientist sta...",Science,"[portrait, scientist, chemist, chemistry, micr...",https://www.shutterstock.com/image-photo/10824...,https://image.shutterstock.com/image-photo/clo...
7,DNA strand and Cancer Cell Oncology Research C...,"Healthcare/Medical,Objects","[cancer, oncology, cell, research, dna, lympho...",https://www.shutterstock.com/image-photo/65963...,https://image.shutterstock.com/image-illustrat...
8,"Young biologist with protective mask, gloves a...","Healthcare/Medical,Industrial","[biologist, biotech, analysis, analyzing, bact...",https://www.shutterstock.com/image-photo/18995...,https://image.shutterstock.com/image-photo/you...
9,Messenger RNA or mRNA strand 3D rendering illu...,"Science,Healthcare/Medical","[mrna, rna, acid, ribonucleic, vaccine, enzyme...",https://www.shutterstock.com/image-photo/19076...,https://image.shutterstock.com/image-illustrat...


In [ ]:
# Save as csv
scraped_df.to_csv("bucket_3.1_shutterstock_scraped.csv")